In [5]:
import torch
import scipy
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split
import sys

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        #kaggle 참조후 변환
        #C1 (convolutional)= 6 feature map(28x28), check 5x5, input image channel =1-grayscal images, 156 trainable parameter, 122,304 connection
        #nn.Conv2d(batch_size, channels(==색상채널), height, width(==이미지 크기), kernel_size=, padding=)
        #nn.Conv2d(in_channels=input channel num, out_channels=out channel num, kernel_size=check neighbors, 
        self.c1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(5, 5), padding='same')
        self.relu1 = nn.ReLU()#activation
        
        #S2 (subsampling)= 6 feature map (14x14), kernel size 2x2, stride=2 default, 12 trainable parameter, 5,880 connection
        #nn.MaxPool2d(kernel_size= ,stride=)    nn.AvgPool2d(kernel_size=, stride=)
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        #C3 (convolutional)= 16 feature maps, kernel size(5x5), 
        self.c3 = nn.Conv2d(in_channels=32, out_channels=48, kernel_size=(5, 5), padding='valid')
        self.relu2 = nn.ReLU()#activation

        #S4 (subsampling)= 16 feature maps(5x5), kernel size(2x2) #stride=2 default
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        #C5 (convolutional)= 120 feature maps, kernel size(5x5), 45 120 trainable connections
        self.c5 = nn.Conv2d(in_channels=48, out_channels=120, kernel_size=(5,5))
        self.relu3 = nn.ReLU()
       
        #F6 (fully ..)=84 units, (fully connected to C5), 10,164 trainable parameter
        self.f6 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()

        #F7
        self.f7 = nn.Linear(84, 10)  # 클래스 수 10개
        self.softmax1 = nn.Softmax(dim=1)

        '''
        #kernel(5) filter가 feature extract. 4x4작아짐
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.f6 = nn.Linear(in_features=120, out_features=84)
        self.f7 = nn.Linear(in_features=84, out_features=10)
        '''

    def forward(self, x):
        x=self.c1(x)
        x=self.s2(x)
        x=self.c3(x)
        x=self.s4(x)
        x=self.c5(x).view(-1,120)#flatten
        x=self.f6(x)
        x=self.f7(x)
        return x

class MyCustomDataset(Dataset):
    def __init__(self):
        dtable = scipy.io.loadmat('MNIST.mat')
        self.data = np.transpose(dtable['data']).astype(np.float32)
        self.label = np.transpose(dtable['label']).squeeze()  # 1D로 변경

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx].reshape(1, 28, 28)
        label = torch.tensor(self.label[idx].astype(np.int64), dtype=torch.long)
        return torch.tensor(data), label

# Define your model
model = MyModel()
train_dataset, valid_dataset = random_split(MyCustomDataset(), [60000, 10000])
train_loader, valid_loader = DataLoader(train_dataset, 512, shuffle=True), DataLoader(valid_dataset, 512, shuffle=False)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-4)

# Set device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
running_loss = 0.0
step = 0
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        model.train()  # Set model to training mode
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = loss_fn(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        step += 1

        # Log training status
        if step % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], '
                  f'Loss: {loss.item():.4f}')

            with torch.no_grad():
                valid_loss = 0.0
                correct = 0
                total = 0
                for inputs, targets in valid_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    valid_loss += loss.item()

                    # For classification tasks, calculate accuracy
                    _, predicted = torch.max(outputs.data, 1)
                    total += targets.size(0)
                    correct += (predicted == targets).sum().item()

                avg_valid_loss = valid_loss / len(valid_loader)
                accuracy = 100 * correct / total
                print(f'Validation Loss: {avg_valid_loss:.4f}, Accuracy: {accuracy:.2f}%\n')


Epoch [1/10], Step [100/118], Loss: 0.9506
Validation Loss: 0.8107, Accuracy: 77.11%

Epoch [2/10], Step [82/118], Loss: 0.6521
Validation Loss: 0.5913, Accuracy: 83.19%

Epoch [3/10], Step [64/118], Loss: 0.4913
Validation Loss: 0.4861, Accuracy: 86.00%

Epoch [4/10], Step [46/118], Loss: 0.3244
Validation Loss: 0.4400, Accuracy: 87.45%

Epoch [5/10], Step [28/118], Loss: 0.4709
Validation Loss: 0.3950, Accuracy: 88.68%

Epoch [6/10], Step [10/118], Loss: 0.3236
Validation Loss: 0.3677, Accuracy: 89.50%

Epoch [6/10], Step [110/118], Loss: 0.3954
Validation Loss: 0.3460, Accuracy: 90.18%

Epoch [7/10], Step [92/118], Loss: 0.2027
Validation Loss: 0.3311, Accuracy: 90.36%

Epoch [8/10], Step [74/118], Loss: 0.3064
Validation Loss: 0.3123, Accuracy: 91.14%

Epoch [9/10], Step [56/118], Loss: 0.3266
Validation Loss: 0.2981, Accuracy: 91.51%

Epoch [10/10], Step [38/118], Loss: 0.4094
Validation Loss: 0.2875, Accuracy: 91.64%



In [ ]:
        #nn.Linear(input 차원, output vector차원)
        #nn.Conv1d(batch_size, sequence_length, channels(==input의 feautre num))

In [17]:
#test
import torch
import scipy.io
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split
import sys

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        #kaggle 참조후 변환
        #C1 (convolutional)= 6 feature map(28x28), check 5x5, input image channel =1-grayscal images, 156 trainable parameter, 122,304 connection
        #nn.Conv2d(batch_size, channels(==색상채널), height, width(==이미지 크기), kernel_size=, padding=)
        #nn.Conv2d(in_channels=input channel num, out_channels=out channel num, kernel_size=check neighbors, 
        self.c1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(5, 5), padding='same')
        self.relu1 = nn.ReLU()#activation
        
        #S2 (subsampling)= 6 feature map (14x14), kernel size 2x2, stride=2 default, 12 trainable parameter, 5,880 connection
        #nn.MaxPool2d(kernel_size= ,stride=)    nn.AvgPool2d(kernel_size=, stride=)
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        #C3 (convolutional)= 16 feature maps, kernel size(5x5), 
        self.c3 = nn.Conv2d(in_channels=32, out_channels=48, kernel_size=(5, 5), padding='valid')
        self.relu2 = nn.ReLU()#activation

        #S4 (subsampling)= 16 feature maps(5x5), kernel size(2x2) #stride=2 default
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        #C5 (convolutional)= 120 feature maps, kernel size(5x5), 45 120 trainable connections
        self.c5 = nn.Conv2d(in_channels=48, out_channels=120, kernel_size=(5,5))
        self.relu3 = nn.ReLU()
       
        #F6 (fully ..)=84 units, (fully connected to C5), 10,164 trainable parameter
        self.f6 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()

        #F7
        self.f7 = nn.Linear(84, 10)  # 클래스 수 10개
        self.softmax1 = nn.Softmax(dim=1)

        '''
        #kernel(5) filter가 feature extract. 4x4작아짐
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.f6 = nn.Linear(in_features=120, out_features=84)
        self.f7 = nn.Linear(in_features=84, out_features=10)
        '''

    def forward(self, x):
        x=self.c1(x)
        x=self.s2(x)
        x=self.c3(x)
        x=self.s4(x)
        x=self.c5(x).view(-1,120)#flatten
        x=self.f6(x)
        x=self.f7(x)
        return x


class MyCustomDataset(Dataset):
    def __init__(self):
        """
        Args:
            data_list (list): List of data samples (e.g., file paths or data arrays).
            targets (list or array, optional): Corresponding labels or targets for supervised learning.
            transform (callable, optional): A function/transform to apply to each sample.
        """
        dtable = scipy.io.loadmat('MNIST.mat')
        self.data = np.transpose(dtable['data']).astype(np.float32)
        self.label = np.transpose(dtable['label'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx].reshape(1,28,28)
        label = torch.tensor(self.label[idx], dtype=torch.long).squeeze()
        return data, label

# Define your model (replace 'MyModel' with your actual model class)
model = MyModel()
train_dataset, valid_dataset = random_split(MyCustomDataset(), [60000, 10000])
train_loader, valid_loader = DataLoader(train_dataset, 512, shuffle=True), DataLoader(train_dataset, 512, shuffle=False)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Replace with your chosen loss function
optimizer = optim.SGD(model.parameters(), lr=1e-4)  # Replace with your optimizer and hyperparameters

# Set device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
running_loss = 0.0
step = 0
num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        model.train()  # Set model to training mode
        # Move data to the appropriate device
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = loss_fn(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        step += 1

        # Log training status
        if step%100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], '
                  f'Loss: {loss.item():.4f}')

            with torch.no_grad():  # Disable gradient computation
                valid_loss = 0.0
                correct = 0
                total = 0
                for inputs, targets in valid_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    valid_loss += loss.item()

                    # For classification tasks, calculate accuracy
                    _, predicted = torch.max(outputs.data, 1)
                    total += targets.size(0)
                    correct += (predicted == targets).sum().item()

            avg_valid_loss = valid_loss / len(valid_loader)
            accuracy = 100 * correct / total
            print(f'Validation Loss: {avg_valid_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

Epoch [1/10], Step [100/118], Loss: 0.8891
Validation Loss: 0.8603, Accuracy: 75.36%

Epoch [2/10], Step [82/118], Loss: 0.5239
Validation Loss: 0.6255, Accuracy: 82.25%

Epoch [3/10], Step [64/118], Loss: 0.5534
Validation Loss: 0.5272, Accuracy: 84.98%

Epoch [4/10], Step [46/118], Loss: 0.3427
Validation Loss: 0.4707, Accuracy: 86.62%

Epoch [5/10], Step [28/118], Loss: 0.4264
Validation Loss: 0.4292, Accuracy: 87.88%

Epoch [6/10], Step [10/118], Loss: 0.3974
Validation Loss: 0.3975, Accuracy: 88.69%

Epoch [6/10], Step [110/118], Loss: 0.3432
Validation Loss: 0.3722, Accuracy: 89.37%

Epoch [7/10], Step [92/118], Loss: 0.4481
Validation Loss: 0.3511, Accuracy: 90.02%

Epoch [8/10], Step [74/118], Loss: 0.3108
Validation Loss: 0.3332, Accuracy: 90.50%

Epoch [9/10], Step [56/118], Loss: 0.3172
Validation Loss: 0.3170, Accuracy: 90.92%

Epoch [10/10], Step [38/118], Loss: 0.2169
Validation Loss: 0.3045, Accuracy: 91.32%



In [20]:
#original paper
import torch
import scipy.io
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split
import sys

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=3)
        self.f6 = nn.Linear(480, 84)
        self.f7 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.c1(x)
        x = self.s2(x)
        x = self.c3(x)
        x = self.s4(x)
        x = self.c5(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.f6(x)
        x = self.f7(x)
        return x


class MyCustomDataset(Dataset):
    def __init__(self):
        """
        Args:
            data_list (list): List of data samples (e.g., file paths or data arrays).
            targets (list or array, optional): Corresponding labels or targets for supervised learning.
            transform (callable, optional): A function/transform to apply to each sample.
        """
        dtable = scipy.io.loadmat('MNIST.mat')
        self.data = np.transpose(dtable['data']).astype(np.float32)
        self.label = np.transpose(dtable['label'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx].reshape(1,28,28)
        label = torch.tensor(self.label[idx], dtype=torch.long).squeeze()
        return data, label

# Define your model (replace 'MyModel' with your actual model class)
model = MyModel()
train_dataset, valid_dataset = random_split(MyCustomDataset(), [60000, 10000])
train_loader, valid_loader = DataLoader(train_dataset, 512, shuffle=True), DataLoader(train_dataset, 512, shuffle=False)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Replace with your chosen loss function
optimizer = optim.SGD(model.parameters(), lr=1e-4)  # Replace with your optimizer and hyperparameters

# Set device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
running_loss = 0.0
step = 0
num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        model.train()  # Set model to training mode
        # Move data to the appropriate device
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = loss_fn(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        step += 1

        # Log training status
        if step%100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], '
                  f'Loss: {loss.item():.4f}')

            with torch.no_grad():  # Disable gradient computation
                valid_loss = 0.0
                correct = 0
                total = 0
                for inputs, targets in valid_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    valid_loss += loss.item()

                    # For classification tasks, calculate accuracy
                    _, predicted = torch.max(outputs.data, 1)
                    total += targets.size(0)
                    correct += (predicted == targets).sum().item()

            avg_valid_loss = valid_loss / len(valid_loader)
            accuracy = 100 * correct / total
            print(f'Validation Loss: {avg_valid_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

Epoch [1/10], Step [100/118], Loss: 1.4594
Validation Loss: 1.6158, Accuracy: 58.92%

Epoch [2/10], Step [82/118], Loss: 1.1554
Validation Loss: 1.0690, Accuracy: 70.20%

Epoch [3/10], Step [64/118], Loss: 0.8509
Validation Loss: 0.8609, Accuracy: 75.65%

Epoch [4/10], Step [46/118], Loss: 0.6452
Validation Loss: 0.7479, Accuracy: 78.69%

Epoch [5/10], Step [28/118], Loss: 0.5645
Validation Loss: 0.6718, Accuracy: 80.83%

Epoch [6/10], Step [10/118], Loss: 0.5740
Validation Loss: 0.6215, Accuracy: 82.18%

Epoch [6/10], Step [110/118], Loss: 0.6133
Validation Loss: 0.5777, Accuracy: 83.38%

Epoch [7/10], Step [92/118], Loss: 0.5791
Validation Loss: 0.5432, Accuracy: 84.31%

Epoch [8/10], Step [74/118], Loss: 0.5097
Validation Loss: 0.5160, Accuracy: 85.03%

Epoch [9/10], Step [56/118], Loss: 0.4300
Validation Loss: 0.4933, Accuracy: 85.67%

Epoch [10/10], Step [38/118], Loss: 0.4502
Validation Loss: 0.4725, Accuracy: 86.16%

